In [24]:
%pwd
import os 
os.chdir(r"c:\vs\ml\chest-cancer-classification-using-mlflow-dvc")
%pwd

'c:\\vs\\ml\\chest-cancer-classification-using-mlflow-dvc'

In [25]:
import sys
from pathlib import Path

# Add project src directory to Python path
project_root = Path(r"c:\vs\ml\chest-cancer-classification-using-mlflow-dvc\src")
sys.path.append(str(project_root))

connecting to dagshub

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/username/repo_name.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="username"
os.environ["MLFLOW_TRACKING_PASSWORD"]="token"

In [27]:
#entity
from dataclasses import dataclass
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [28]:
import tensorflow as tf
from cnnClassifier.utils.common import read_yaml,create_directories,save_json
from cnnClassifier.constants import *


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        # Updated path for your test data
        test_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data", "test")
        
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path(test_data),
            mlflow_uri=os.environ["MLFLOW_TRACKING_URI"],
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [29]:
#component
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse #To check if MLflow is using a remote server (DagsHub) or local file storage ,, Imports a function to parse URL
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="ChestCancerVGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [30]:
#pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    
    print(f"\n✓ Evaluation Complete!")
    print(f"Loss: {evaluation.score[0]:.4f}")
    print(f"Accuracy: {evaluation.score[1]:.4f}")
    
except Exception as e:
    raise e

[2025-10-05 00:36:03,185: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-05 00:36:03,189: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-05 00:36:03,191: INFO: common: created directory at: artifacts]
[2025-10-05 00:36:03,486: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 315 images belonging to 4 classes.
20/20 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.4127 - loss: 1.1942
[2025-10-05 00:37:27,740: INFO: common: json file saved at: scores.json]


2025/10/05 00:37:29 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/10/05 00:38:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ChestCancerVGG16Model' already exists. Creating a new version of this model...
2025/10/05 00:38:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ChestCancerVGG16Model, version 3
Created version '3' of model 'ChestCancerVGG16Model'.


🏃 View run loud-cub-645 at: https://dagshub.com/akakak3188/Chest-Cancer-Classification-using-MLflow-DVC.mlflow/#/experiments/0/runs/90fd5ec16a6c4821928269998a3f444a
🧪 View experiment at: https://dagshub.com/akakak3188/Chest-Cancer-Classification-using-MLflow-DVC.mlflow/#/experiments/0

✓ Evaluation Complete!
Loss: 1.1942
Accuracy: 0.4127
